# Week 4 Extending SQL by PostgreSQL

## Ways that SQL could be extended

* To accurately express the “business logic“ of withdrawing money, we need facilities like conditional controls

## A Stored Procedure Approach

## SQL/PSM


## PostgreSQL

A PostgreSQL-specific language integrating features of:
* procedural programming (PL)
* SQL programming

### User-defined Data Types
PostgreSQL also provides mechanisms to define new types
* basic types: CREATE DOMAIN
* tuple types: CREATE TYPE

> Example:
> 
> Create Domain UnswCourseCode as text
> 
> check (value ~ '[A-Z]{4}[0-9]{4}' );

### PostgreSQL: SQL Functions

```
CREATE OR REPLACE FUNCTION
funcName(arg1type, arg2type, ....)
RETURNS rettype
AS $$
SQL statements
$$ LANGUAGE sql;
```

Example:
```
-- max price of specified beer
create or replace function
    maxPrice(text) returns float
as $$
    select max(price) from Sells where beer = $1;
$$ language sql;
```
using positional notation ($1, $2, ...)

### PL/pgSQL Function Parameters

```
    CREATE OR REPLACE FUNCTION
        funcName(param1, param2, ....)
    RETURNS rettype
    AS $$
    DECLARE
        variable declarations
    BEGIN
        code for function
    END;
    $$ LANGUAGE plpgsql;
```

> Note: the entire function body is a single SQL string

Example:
```
    CREATE OR REPLACE FUNCTION
        add(x text, y text) RETURNS text
    AS $$
    DECLARE
        result text; -- local variable
    BEGIN
        result := x||''''||y;
        return result;
    END;
    $$ LANGUAGE plpgsql;
```

### Using PL/pgSQL Functions
* As part of a SELECT statement

```
select myFunction ( arg1 , arg2 );
select * from myTableFunction ( arg1 , arg2 );
```

* As part of the execution of another PLpgSQL function
```
PERFORM myVoidFunction ( arg1 , arg2 );
result := myOtherFunction ( arg1 );

```
* Automatically, via an insert/delete/update trigger

```
create trigger T before update on R
for each row execute procedure myCheck ();
```

## Declaring Data Types

```
account Accounts%ROWTYPE ;
```

```
account.branchName%TYPE
```

## Control Structures in PL/pgSQL

### Assignment 
variable := expression;

Example:
```
    tax := subtotal * 0.06;
    my_record.user_id := 20;
```

### Conditionals
* IF ... THEN
* IF ... THEN ... ELSE
* IF ... THEN ... ELSIF ... THEN ... ELSE
Example:

```
IF v_user_id > 0 THEN
UPDATE users SET email = v_email WHERE user_id =
v_user_id; END IF;
```


```
create function
        withdraw(acctNum text, amount integer) returns text as $$
declare bal integer;
begin
        select balance into bal
        from Accounts
        where acctNo = acctNum;
        if (bal < amount) then
                return 'Insufficient Funds';
        else
                update Accounts
                set balance = balance - amount
                where acctNo = acctNum;
                select balance into bal
                from Accounts where acctNo = acctNum;
                return 'New Balance: ' || bal;
        end if;
end;
$$ language plpgsql;
```

## SELECT ... INTO